# Lab | Revisiting Machine Learning Case Study

In this lab, you will use learningSet.csv file which you already have cloned in today's activities.

In [2]:
import pandas as pd
import numpy as np
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('learningSet.csv')
data

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,,,0,0,,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,,,5001,0,,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,,,3801,0,,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,,,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


### 1. Check for null values in all the columns

In [4]:
nulls = pd.DataFrame(data.isna().sum()*100/len(data), columns=['percentage'])
nulls.sort_values('percentage', ascending = False).head(80)

,percentage
RDATE_5,99.990567
RAMNT_5,99.990567
RDATE_3,99.746363
RAMNT_3,99.746363
RDATE_4,99.705488
...,...
ADATE_16,21.343227
ADATE_14,19.774242
ADATE_9,11.785729
ADATE_11,10.923154


In [5]:
drop_col= list(nulls[nulls['percentage'] > 10].index)

In [6]:
new_data = data.drop(columns=drop_col, axis=1)

### 2. Exclude the following variables by looking at the definitions. Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

- `OSOURCE` - symbol definitions not provided, too many categories
- `ZIP CODE` - we are including state already


In [7]:
drop_list = new_data.drop(['ZIP', 'OSOURCE'], axis=1)

### 3. Identify columns that over 85% missing values

In [8]:
null_percent_data = data.isna().sum().sort_values(ascending = False)*100/len(data)
null_percent_data = null_percent_data.reset_index()

In [9]:
null_percent_data.columns = ['column_name', 'null_percent']
columns_above_85 = null_percent_data[null_percent_data['null_percent'] > 15]
columns_above_85

,column_name,null_percent
0,RDATE_5,99.990567
1,RAMNT_5,99.990567
2,RDATE_3,99.746363
3,RAMNT_3,99.746363
4,RDATE_4,99.705488
...,...,...
72,AGE,24.802960
73,INCOME,22.309563
74,ADATE_18,22.285457
75,ADATE_16,21.343227


### 4. Remove those columns from the dataframe

In [10]:
for i in columns_above_85['column_name']:
    drop_list.append(i)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

### 5. Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest

Note that there are a few null values in the column. We will first replace those null values using the code below:

```python
print(categorical['GENDER'].value_counts())
categorical['GENDER'] = categorical['GENDER'].fillna('F')
```